In [15]:
import numpy as np
import pandas as pd
import warnings
import os
from tqdm import tqdm
from sklearn import preprocessing, metrics
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
from joblib import Parallel, delayed

%matplotlib inline

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

warnings.filterwarnings('ignore')

In [16]:
next_action = pd.read_csv('./temp/next_action.csv')


In [17]:
next_action.head()

,id
0,150310
1,32147
2,189309
3,17842
4,46490


In [18]:
df_feature = pd.read_pickle('./temp/part1_speicial_feature.plk')

print(df_feature.shape)
df_feature = next_action.merge(df_feature, how='left')
print(df_feature.shape)
df_feature.head(100)

(150060, 35)
(48251, 35)


,id,courier_id,wave_index,tracking_id,courier_wave_start_lng,courier_wave_start_lat,action_type,expect_time,date,type,target,group,current_time,last_tracking_id,last_action_type,source_lng,source_lat,target_lng,target_lat,grid_distance,TA,aoi_id,shop_id,promise_deliver_time,estimate_pick_time,assigned_time,speed,max_load,weather_grade,level,target_MHD,promise_dif,estimate_dif,Restime_dif,assign_dif
0,150310,10290949,5,2100075633932043269,121.411024,39.182940,0,1582538861,20200224,train,1.0,33303,1582538560,2100075631583036554,0,121.416654,39.203351,121.417974,39.201901,958.0,105637,14639,7765,1582540148,1582537688,1582537174,0.717700,0.544444,0.100000,1.100000,0.131356,1588,-872,2460,-1386
1,32147,10298963,1,2100074804874806590,121.531926,39.250766,0,1581051114,20200207,train,1.0,7443,1581050821,2100074805553628748,1,121.526993,39.260214,121.527484,39.261721,549.0,23300,24807,1794,1581052142,1581050283,1581049640,0.554962,0.655556,1.100000,1.100000,0.122617,1321,-538,1859,-1181
2,189309,117054417,0,2100075865005852254,121.449011,39.094955,1,1582857411,20200228,train,1.0,41544,1582857111,2100075865747752238,0,121.437904,39.097161,121.439220,39.100178,616.0,124460,6151,8358,1582863668,1582855928,1582855297,0.686344,0.655556,0.100000,1.100000,0.149050,6557,-1183,7740,-1814
3,17842,117026449,2,2100074692477486783,121.589964,39.097858,1,1580800229,20200204,train,1.0,4260,1580800058,2100074670098384550,1,121.569121,39.101864,121.573177,39.099671,631.0,11946,141,7926,1580801465,1580800506,1580799089,0.553453,0.488889,0.100000,1.100000,0.170739,1407,448,959,-969
4,46490,107080018,4,2100074903132964329,121.520781,39.108080,0,1581253423,20200209,train,1.0,10446,1581252648,2100074904879007269,0,121.507304,39.116371,121.517294,39.090871,4063.0,33727,15069,3664,1581254349,1581252669,1581251863,0.547291,0.544444,0.433333,1.100000,0.501748,1701,21,1680,-785
5,122278,117672601,3,2100075377961632809,121.756910,39.106289,1,1582181803,20200220,train,1.0,27003,1582181396,2100075380727056140,1,121.760763,39.111586,121.763147,39.111971,374.0,81939,23046,10419,1582189799,1582181700,1582180804,0.868157,0.544444,0.100000,1.100000,0.131345,8403,304,8099,-592
6,29252,117008849,0,2100074756835182011,121.444024,39.092173,0,1580958427,20200206,train,1.0,6739,1580958002,2100074758751422420,0,121.445794,39.102141,121.441374,39.111651,1908.0,17845,8470,10005,1580958900,1580955605,1580956476,0.267298,0.488889,1.100000,1.100000,0.257688,898,-2397,3295,-1526
7,145169,106582138,6,2100075574267184703,121.492217,39.173475,0,1582453674,20200223,train,1.0,31868,1582453433,2100075574271149631,1,121.484190,39.165002,121.486024,39.167621,931.0,100536,2355,206,1582453755,1582452615,1582451756,0.622684,0.433333,0.100000,0.766667,0.150408,322,-818,1140,-1677
8,14514,10007871,4,2100074700228298547,121.640102,39.139289,1,1580810618,20200204,train,1.0,3498,1580810402,2100074701416334866,1,121.629554,39.154411,121.626703,39.151882,634.0,12644,26495,2170,1580813072,1580811272,1580810234,0.562637,0.655556,0.100000,1.100000,0.160902,2670,870,1800,-168
9,188725,115753441,6,2100075924648363068,121.542211,39.321219,1,1582888756,20200228,train,1.0,41421,1582888753,2100075922554422332,1,121.526414,39.334894,121.526414,39.334894,1.0,129854,21224,124,1582891530,1582888710,1582887673,0.948778,0.544444,0.100000,1.100000,0.100000,2777,-43,2820,-1080


In [19]:
df_feature['type'].value_counts()

train    44058
test      4193
Name: type, dtype: int64

In [20]:
df_feature.head()

,id,courier_id,wave_index,tracking_id,courier_wave_start_lng,courier_wave_start_lat,action_type,expect_time,date,type,target,group,current_time,last_tracking_id,last_action_type,source_lng,source_lat,target_lng,target_lat,grid_distance,TA,aoi_id,shop_id,promise_deliver_time,estimate_pick_time,assigned_time,speed,max_load,weather_grade,level,target_MHD,promise_dif,estimate_dif,Restime_dif,assign_dif
0,150310,10290949,5,2100075633932043269,121.411024,39.182940,0,1582538861,20200224,train,1.0,33303,1582538560,2100075631583036554,0,121.416654,39.203351,121.417974,39.201901,958.0,105637,14639,7765,1582540148,1582537688,1582537174,0.717700,0.544444,0.100000,1.1,0.131356,1588,-872,2460,-1386
1,32147,10298963,1,2100074804874806590,121.531926,39.250766,0,1581051114,20200207,train,1.0,7443,1581050821,2100074805553628748,1,121.526993,39.260214,121.527484,39.261721,549.0,23300,24807,1794,1581052142,1581050283,1581049640,0.554962,0.655556,1.100000,1.1,0.122617,1321,-538,1859,-1181
2,189309,117054417,0,2100075865005852254,121.449011,39.094955,1,1582857411,20200228,train,1.0,41544,1582857111,2100075865747752238,0,121.437904,39.097161,121.439220,39.100178,616.0,124460,6151,8358,1582863668,1582855928,1582855297,0.686344,0.655556,0.100000,1.1,0.149050,6557,-1183,7740,-1814
3,17842,117026449,2,2100074692477486783,121.589964,39.097858,1,1580800229,20200204,train,1.0,4260,1580800058,2100074670098384550,1,121.569121,39.101864,121.573177,39.099671,631.0,11946,141,7926,1580801465,1580800506,1580799089,0.553453,0.488889,0.100000,1.1,0.170739,1407,448,959,-969
4,46490,107080018,4,2100074903132964329,121.520781,39.108080,0,1581253423,20200209,train,1.0,10446,1581252648,2100074904879007269,0,121.507304,39.116371,121.517294,39.090871,4063.0,33727,15069,3664,1581254349,1581252669,1581251863,0.547291,0.544444,0.433333,1.1,0.501748,1701,21,1680,-785


In [21]:
df_feature.to_pickle('./temp/part2_feature.plk')